In [2]:
from bokeh.plotting import figure,ColumnDataSource, show
from bokeh.models import Legend
from bokeh.io import output_notebook
from bokeh.palettes import HighContrast3,TolRainbow11
from bokeh.plotting import  ColumnDataSource
from lib.figures import *
from lib.constant import gp_politique_color


import numpy as np
output_notebook() #To display figure in notebook # needed for jupyterbook

Loading BokehJS ...

In [3]:
import pandas as pd
twitter_df = pd.read_parquet('data/twitter_fev_to_juin_2023_retraite_data.parquet')

In [4]:

def getCountDataframe(df: pd.DataFrame, top_n: int = 20):

    selected_keywords =[]
    for gram in range(1,4):
        for gp_politique in df.groupe_sigle.unique():
            temp = getCountKeywordsByPoliticalGroup(df,gp_politique)
            temp = temp[temp.num_words == gram]
            selected_keywords.extend(temp.term.head(top_n).values.tolist())
    selected_keywords = np.unique(selected_keywords)

    count_df = []
    for gram in range(1,4):
        for gp_politique in df.groupe_sigle.unique():
            temp: pd.DataFrame = getCountKeywordsByPoliticalGroup(df, gp_politique)
            temp = temp[temp.num_words == gram]
            temp = temp[temp.term.isin(selected_keywords)]
            temp["groupe_politique"] = gp_politique
            count_df.append(temp)

    count_df = pd.concat(count_df)
    
    count_df["percentage"] = count_df.\
        apply(lambda x: (x.freq/count_df[count_df.term == x.term].freq.sum())*100, axis=1)
    return count_df

In [5]:
df = getCountDataframe(twitter_df)
df

,term,freq,num_words,groupe_politique,percentage
12,loi,3877,1,REN,26.567532
14,France,2147,1,REN,28.083715
6,travail,1761,1,REN,30.221383
21,français,1616,1,REN,21.770174
41,pouvoir,1561,1,REN,22.367101
...,...,...,...,...,...
266,taux de emploi,1,3,LR,3.448276
260,étude de impact,1,3,LR,4.166667
254,santé au travail,1,3,LR,3.448276
38,financer le retraite,1,3,LR,1.960784


In [6]:
from pprint import pprint
from lib.constant import gp_politique_color
from bokeh.models import Legend

In [12]:
def bokeh_fig(df):
    terms = df.term.unique().tolist()
    groupes_politiques = df.groupe_politique.unique().tolist()

    data = {"term":terms}
    for gp_politique in groupes_politiques:
        percentage = []
        for term in terms:
            try:
                x = df[(df.term == term) & (df.groupe_politique == gp_politique)].iloc[0].percentage
            except:
                x=0
            percentage.append(x)
        data[gp_politique] = percentage


    p = figure(y_range=terms, height=500,title="Distribution des occurrences de top mots-clés",toolbar_location="above",
            tools="hover", tooltips="@term($name) = @$name%",sizing_mode="stretch_width")
    rs = p.hbar_stack(groupes_politiques, y='term', line_color="#444",width=0.9, color=[gp_politique_color[gp] for gp in groupes_politiques], source=data)

    p.x_range.start = 0
    p.x_range.end = 100
    p.xgrid.grid_line_color = None


    legend = Legend(items=[(fruit, [r]) for (fruit, r) in zip(groupes_politiques, rs)], location="center")
    p.add_layout(legend, 'right')

    return p

In [13]:
from bokeh.models import TabPanel, Tabs

tab1 = TabPanel(child=bokeh_fig(df[df.num_words == 1]), title="1 mot")
tab2 = TabPanel(child=bokeh_fig(df[df.num_words == 2]), title="2 mots")
tab3 = TabPanel(child=bokeh_fig(df[df.num_words == 3]), title="3 mots")

show(Tabs(tabs=[tab1, tab2,tab3],width=1000))